# Service Ticket Intelligence Recommendation 

In this notebook, we will see how to prepare the data for recommendation, upload the data, start training and do inference.

### Install pyjwt library if not already installed

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib.pyplot as plt
import requests
import base64

### Load and prepare data

We have a small dataset of questions asked in Quora. We will attempt to build a recommendation model that will recommend similiar questions if we get similar questions (e.g. worded differently, but semantically have same meaning). In the same way, for new incoming service tickets, similar tickets can be recommended to service agents to aid in faster resolution. 

The below code block loads the data from file

In [ ]:
df = pd.read_csv("../datasets/quora.csv")

### Let's see the data

In [ ]:
df.head()

### Let's select the input and output mappings for training

The mapping describes which columns in the upload file should be used as sample input and which ones are to be used as the classification output that the model should learn.

In [ ]:
input_cols = ['subject']
output_cols = ['subject', 'article_id', 'article_desc', 'article_url']
all_cols = input_cols + output_cols

# STI REST Endpoints

The STI service can be accessed and controlled through REST endpoint.
Documentation can be found in the following link: https://help.sap.com/viewer/product/SERVICE_TICKET_INTELLIGENCE

## Subscription and Authentication

Now we are ready to train a model using the Service Ticket Intelligence API. This requires a valid subscription to the STI API.

Note: Update the values for `service url`, `uaa url`, `client id` and `client secret` in the config file `sti_config.ini`. This config file is placed one directory above this notebook. These values will be available in `service_keys` of your STI instance in the cloud foundry cockpit.

In [ ]:
import configparser
from pathlib import Path
import sys

sys.path.append("..")
from sti_functions import STIFunctions, get_connection_object

In [ ]:
STI_BASE_DIR = Path.cwd().parent
config_file_path = STI_BASE_DIR / 'default_key.json'

connection = get_connection_object(config_file=config_file_path)
sti = STIFunctions(connection)

## List models

Now lets do list model call using this python function to view all the models in this account

In [ ]:
sti.list_models()

### Let's check if we need to delete any unused model
Based on the model list above, ensure that the number of models does not exceed 20. Otherwise, we need to delete some unused model.

In [ ]:
# sti.delete_model("b3e0da989393450ba4b66e637722034a")

## File upload

This process will take a few minutes to complete depending on the file size. If file upload is successful, the response text will contain a model id - an UUID identifier which we can use as a reference to the uploaded training file.

In [ ]:
payload = {
    "scenario": {
        "desc": "Quora data for recommendation",
        "type": "recommendation",
        "language": "en",
        "business_object": "ticket",
    },
    "mapping": {
        "input": input_cols,
        "output": output_cols
    },
    "training": {
        "file": "{}".format(
            base64.b64encode(df.to_csv(index=False).encode("utf-8")).decode("utf-8")
        )
    },
}
response = sti.file_upload(payload)
payload = {}
our_model_id = response.get("model_id")
response

## Start training on uploaded file

Take the model id from file upload response text and pass it when in starting the model training

In [ ]:
# our_model_id = "e44816c7732944288d85b890373eb5f4"
sti.start_model_training(model_id=our_model_id)

## Wait for training to succeed

After starting the model training, do a get model status and check if model status is `READY`

The model status transitions from `NEW` to `PENDING_TRAINING` once training is submitted and will further transition to `IN_TRAINING` and finally `READY` when training succeeds

In [ ]:
status = sti.get_model_status(model_id=our_model_id)
print("Model status: {}".format(status.get('model_status')))

Wait for model status to be `READY` before proceeding to next step. This will take upto 10-20 mins from the training submission time. Repeatedly run the above cell to get the latest model status

Once the model status is `READY` proceed to next step.

## Model recall rate

As this is an unsupervised model, we will retrieve the recall rate once training is completed and status becomes ready.

In [ ]:
# our_model_id = "cc078a539d6a433a92f0ac0a2fb445d2"
status = sti.get_model_status(model_id=our_model_id)
status

## Activate the model

Once you are satisfied with the results, model needs to activated before inference can be run on

In [ ]:
sti.activate_model(model_id=our_model_id)

## Wait for activation

Wait till model status transition from `READY` to `LOADING` to `ACTIVE`

In [ ]:
status = sti.get_model_status(model_id=our_model_id)
print("Model status: {}".format(status.get('model_status')))

It takes up to 10 - 15 minutes for the model status to transition from `READY` -> `LOADING` -> `ACTIVE`

## Let's send some inference request

Let's pick one of the data from training dataset

In [ ]:
df.iloc[3]

Reword the question and send the inference request. `field` should match that of the input column. Response should closest question which of might be same as our original question

In [ ]:
inference_payload = {
    "business_object": "ticket",
    "messages": [],
    "options": {
        "recommendation_top_k": 2
    }
}
inference_payload['messages'].append(
    {
        'id': 1,
        'contents': [
            {
                'field': 'subject',
                'value': 'How to make physics easy to learn?'
            }
        ]
    }
)
inference_response = sti.recommend(data_payload=inference_payload)
inference_response

## Deactivate model

We can deactivate any active models here.

In [ ]:
# sti.deactivate_model(model_id="")

## Delete model

We can delete any unused models here.

In [ ]:
# sti.delete_model(model_id="")